In [ ]:
import panel as pn
import pandas as pd
import numpy as np
import math
import s3fs
import xarray as xr
import gc

from os import path

import datetime as dt
from bokeh.plotting import figure
from bokeh.layouts import gridplot

pn.extension()

In [ ]:
def castWindow(asset, sampleTime, castWindowDays):
    
    castWindows = []
    for deploy in RefDes_dict[asset]:
        if isinstance(deploy['deployEnd'],str):
            deployEnd = dt.datetime.strptime(deploy['deployEnd'], '%Y-%m-%dT%H:%M:%S')
        elif isinstance(deploy['deployEnd'],float):
            if math.isnan(deploy['deployEnd']):
                deployEnd = dt.datetime.now()      
        if deploy['deployDate'] <= sampleTime <= deployEnd:
            castWindowStart = sampleTime - dt.timedelta(days = 0.5)
            castWindowEnd = sampleTime + dt.timedelta(days = 0.5)
            castWindows.append([castWindowStart,castWindowEnd,deploy['deployNum']])
        elif (deploy['deployDate'] > sampleTime) & ((deploy['deployDate'] - sampleTime).days < castWindowDays):
            castWindowStart = deploy['deployDate']
            castWindowEnd = deploy['deployDate'] + dt.timedelta(days = 1)
            castWindows.append([castWindowStart,castWindowEnd,deploy['deployNum']])
        elif (deployEnd < sampleTime) & ((sampleTime - deployEnd).days < castWindowDays):
            castWindowStart = deployEnd - dt.timedelta(days = 1)
            castWindowEnd = deployEnd
            castWindows.append([castWindowStart,castWindowEnd,deploy['deployNum']])
        else:
            next
            
    return castWindows

In [ ]:
def extractFixed(data,sampleTime_string,window):
    
    sampleTime = np.datetime64(sampleTime_string)
    # if sampleTime within cast window:
    if (data['time'][0].values < (sampleTime - np.timedelta64(int(window/2),'m'))) & (data['time'][-1].values > (sampleTime + np.timedelta64(int(window/2),'m'))):
        extractStart = sampleTime - np.timedelta64(int(window/2),'m')
        extractEnd = sampleTime + np.timedelta64(int(window/2),'m')
    elif (sampleTime > data['time'][-1].values):
        extractStart = data['time'][-1].values - np.timedelta64(window,'m')
        extractEnd = data['time'][-1].values
    elif (sampleTime < data['time'][0].values):
        extractStart = data['time'][0].values
        extractEnd = data['time'][0].values + np.timedelta64(window,'m')

    extractData = data.sel(time=slice(extractStart, extractEnd))

    return(extractData)

In [ ]:
def extractProfiles(data,sampleTime_string,castState):
    
    sampleTime = np.datetime64(sampleTime_string)
   
    if 'seawater_pressure' in data.data_vars:
        pressure = data['seawater_pressure'].values
    elif 'int_ctd_pressure' in data.data_vars:
        pressure = data['int_ctd_pressure'].values
    else:
        print('no valid pressure variable found...')
    
    # todo...how to define pressChange, window, and parking thresholds for different profilers???
    # 0.3, 10, 175 seems to work for an example of CTD at SF01B???
    # 1, 10, 175 seemed to work for an example of CTD at SF03A???
    profileState = findCastIndex(pressure,0.3,10,175)

    if 'UP' in castState:
        castIndex = 10
    elif 'DOWN' in castState:
        castIndex = 5
    elif 'PARK' in castState:
        castIndex = 0
    
    profileIdx = []
    profileTime = []
    profilePressure = []
    for d in range(len(profileState)):
        if profileState[d] == castIndex:
            profileIdx.append(d)
            profileTime.append(data['time'][d].values)
            profilePressure.append(pressure[d])
            
    castIdx = []  
    castTime = []
    castPressure = []
    castEnd = 0
    for e in range(1,len(profileTime)):
        timeDiff = profileTime[e] - profileTime[e-1]
        if ((profileTime[e] - profileTime[e-1]) > np.timedelta64(20,'m')):
            #print('foundStart')
            for f in range(e,len(profileTime)-1):
                if profileTime[f+1] - profileTime[f] > np.timedelta64(20,'m'):
                    #print('foundEnd')
                    castEnd = 1
                    castIdx.append([profileIdx[e],profileIdx[f]])
                    castPressure.append([profilePressure[e],profilePressure[f]])
                    castTime.append([profileTime[e],profileTime[f]])
                    break
            if castEnd == 0:
                castIdx.append([profileIdx[e],profileIdx[-1]])
                castPressure.append([profilePressure[e],profilePressure[-1]])
                castTime.append([profileTime[e],profileTime[-1]])
                break
            
    if len(castTime) > 0:
        cast = min(castTime, key = lambda x: abs(x[0] - sampleTime))
        extractData = data.sel(time=slice(cast[0], cast[1]))
    else:
        extractData = 'NoValidData'
    
    return(extractData)

In [ ]:
def findCastIndex(press,pressChange,windowSize,threshold_parked):
    profileState = np.empty(len(press))
    profileState[:] = np.nan
    #profileState[:] = 20
    profileState = profileState.tolist()

    for i in range(len(press)-windowSize):
        state = press[i] - press[i+windowSize]
        if state > pressChange:
            profileState[i] = 10
        elif state < -pressChange:
            profileState[i] = 5
        elif abs(state) < pressChange and press[i] > threshold_parked:
            profileState[i] = 0
            
    return(profileState)

In [ ]:
### load in discrete summaries

Summaries = pd.Series(['Cabled-6_TN326_Discrete_Summary_2019-10-21_ver_2-00.csv',
                         'Cabled-7_SKQ201610S_Discrete_Summary_2019-10-21_ver_2-00.csv',
                         'Cabled-8_RR1713-RR1717_Discrete_Summary_2019-10-21_ver_2-00.csv',
                         'Cabled-9_RR1809-RR1812_Discrete_Summary_2019-12-06_ver_2-00.csv',
                         'Cabled-10_AT4212_Discrete_Summary_2019-12-19_ver_2-00.csv'])

df_discreteSummaries = pd.concat([pd.read_csv(f,na_values=['-9999999']) for f in Summaries])


In [ ]:
### load in instrument and stream maps
instFile = 'instrumentDictionary.csv'
streamFile = 'streamDictionary_zarr.csv'

stream_list = pd.read_csv(streamFile)
inst_list = pd.read_csv(instFile)

#inst_dict = pd.Series(inst_list['instruments'].values, index=inst_list['site']).to_dict()
inst_dict = inst_list.set_index('site').T.to_dict('series')

stream_list['parameters'] = stream_list['parameters'].str.split(',')

stream_dict = stream_list.set_index('instrument').T.to_dict('series')


In [ ]:
# Load deployment sheets and create reference designator dictionary

amRepo = path.dirname('/Users/rsn/asset-management/')

### load in github deployment sheets
CabledArray = pd.Series(['CE02SHBP','CE04OSBP','CE04OSPD','CE04OSPS','RS01SBPD','RS01SBPS',
                        'RS01SLBS','RS01SUM1','RS03AXBS','RS03AXPD','RS03AXPS','RS03INT2',
                        'RS03INT1','RS01SUM2','RS03CCAL','RS03ECAL','RS03ASHS'])

deploymentSheets = []
for array in CabledArray:
    deployFilePath = amRepo + '/deployment/' + array + '_Deploy.csv' 
    deploymentSheets.append(deployFilePath)
    
df_deploy = pd.concat([pd.read_csv(f, skip_blank_lines = True, comment='#') for f in deploymentSheets], ignore_index = True)

# Deployment sheet Reference Designator key to startDate, AssetID, rawFile
df_deploy_sort = df_deploy.sort_values(by=["Reference Designator","startDateTime"],ascending=False)

RefDes_dict = {}
for i in df_deploy_sort['Reference Designator'].unique():
    RefDes_dict[i] = [{'deployDate':dt.datetime.strptime(df_deploy_sort['startDateTime'][j], '%Y-%m-%dT%H:%M:%S'), 'deployEnd':df_deploy_sort['stopDateTime'][j], 'AssetID':df_deploy_sort['sensor.uid'][j], 'deployNum':df_deploy_sort['deploymentNumber'][j], 'latitude':df_deploy_sort['lat'][j],'longitude':df_deploy_sort['lon'][j]} for j in df_deploy_sort[df_deploy_sort['Reference Designator']==i].index]


In [ ]:
bottleVariables = ['CTD Pressure [db]','CTD Depth [m]','CTD Temperature 1 [deg C]','CTD Temperature 2 [deg C]',
                'CTD Conductivity 1 [S/m]','CTD Conductivity 2 [S/m]','CTD Salinity 1 [psu]',
                'CTD Salinity 2 [psu]','CTD Oxygen [mL/L]','CTD Oxygen Saturation [mL/L]',
                'CTD Fluorescence [mg/m^3]','CTD Beam Attenuation [1/m]','CTD Beam Transmission [%]','CTD pH',
                'Discrete Oxygen [mL/L]','Discrete Chlorophyll [ug/L]','Discrete Phaeopigment [ug/L]',
                'Discrete Fo/Fa Ratio','Discrete Phosphate [uM]','Discrete Silicate [uM]',
                'Discrete Nitrate [uM]','Discrete Nitrite [uM]','Discrete Ammonium [uM]',
                'Discrete Salinity [psu]','Discrete Alkalinity [umol/kg]','Discrete DIC [umol/kg]',
                'Discrete pCO2 [uatm]','Discrete pH [Total scale]','Calculated Alkalinity [umol/kg]',
                'Calculated DIC [umol/kg]','Calculated pCO2 [uatm]','Calculated pH',
                'Calculated CO2aq [umol/kg]','Calculated Bicarb [umol/kg]','Calculated CO3 [umol/kg]',
                'Calculated Omega-C','Calculated Omega-A']
###Niskin/Bottle Position,Niskin Flag,CTD Bottle Closure Time [UTC],CTD Pressure [db],CTD Pressure Flag,CTD Depth [m],CTD Latitude [deg],CTD Longitude [deg],CTD Temperature 1 [deg C],CTD Temperature 1 Flag,CTD Temperature 2 [deg C],CTD Temperature 2 Flag,CTD Conductivity 1 [S/m],CTD Conductivity 1 Flag,CTD Conductivity 2 [S/m],CTD Conductivity 2 Flag,CTD Salinity 1 [psu],CTD Salinity 2 [psu],CTD Oxygen [mL/L],CTD Oxygen Flag,CTD Oxygen Saturation [mL/L],CTD Fluorescence [mg/m^3],CTD Fluorescence Flag,CTD Beam Attenuation [1/m],CTD Beam Transmission [%],CTD Transmissometer Flag,CTD pH,CTD pH Flag,Discrete Oxygen [mL/L],Discrete Oxygen Flag,Discrete Oxygen Replicate Flag,Discrete Chlorophyll [ug/L],Discrete Phaeopigment [ug/L],Discrete Fo/Fa Ratio,Discrete Fluorescence Flag,Discrete Fluorescence Replicate Flag,Discrete Phosphate [uM],Discrete Silicate [uM],Discrete Nitrate [uM],Discrete Nitrite [uM],Discrete Ammonium [uM],Discrete Nutrients Flag,Discrete Nutrients Replicate Flag,Discrete Salinity [psu],Discrete Salinity Flag,Discrete Salinity Replicate Flag,Discrete Alkalinity [umol/kg],Discrete Alkalinity Flag,Discrete Alkalinity Replicate Flag,Discrete DIC [umol/kg],Discrete DIC Flag,Discrete DIC Replicate Flag,Discrete pCO2 [uatm],pCO2 Analysis Temp [deg C],Discrete pCO2 Flag,Discrete pCO2 Replicate Flag,Discrete pH [Total scale],pH Analysis Temp [deg C],Discrete pH Flag,Discrete pH Replicate Flag,Calculated Alkalinity [umol/kg],Calculated DIC [umol/kg],Calculated pCO2 [uatm],Calculated pH,Calculated CO2aq [umol/kg],Calculated Bicarb [umol/kg],Calculated CO3 [umol/kg],Calculated Omega-C,Calculated Omega-A

# build dictionary indexed by cruise, cast, and corresponding deployments of target assets
arrayData = {}
targetAssetsAll = []
for cruise in df_discreteSummaries['Cruise'].unique():
    arrayData.setdefault(cruise,{})
    casts = df_discreteSummaries['Cast'][df_discreteSummaries['Cruise'].str.contains(cruise)].unique()
    for cast in casts:
        arrayData[cruise].setdefault(cast,{})
        arrayData[cruise][cast]['sampleTime'] = df_discreteSummaries['Start Time [UTC]'][df_discreteSummaries['Cruise'].str.contains(cruise) & df_discreteSummaries['Cast'].str.contains(cast)].unique()[0]
        arrayData[cruise][cast]['station'] = df_discreteSummaries['Station'][df_discreteSummaries['Cruise'].str.contains(cruise) & df_discreteSummaries['Cast'].str.contains(cast)].unique()[0]
        castFlag = df_discreteSummaries['Cast Flag'][df_discreteSummaries['Cruise'].str.contains(cruise) & df_discreteSummaries['Cast'].str.contains(cast)].unique()[0]
        ctdFlag = df_discreteSummaries['CTD File Flag'][df_discreteSummaries['Cruise'].str.contains(cruise) & df_discreteSummaries['Cast'].str.contains(cast)].unique()[0]
        arrayData[cruise][cast].setdefault('bottleData',{})
        castType = "uknown"
        #if '-9999999' not in ctdFlag:
        if isinstance(ctdFlag,str):
            if int(ctdFlag[15]) == 1:
                castType = 'dataOnlyCast'    
            else:
                castType = 'bottleCast'
        else:
            if int(castFlag[3]) == 1:
                castType = 'ROVcast'
        arrayData[cruise][cast]['bottleData']['castType'] = castType
        for var in bottleVariables:
            varData = df_discreteSummaries[var][df_discreteSummaries['Cruise'].str.contains(cruise) & df_discreteSummaries['Cast'].str.contains(cast)].values.tolist()
            arrayData[cruise][cast]['bottleData'][var] = varData

        ### find targetAssets based on targetAsset column in discrete summary:
        targetSites = df_discreteSummaries['Target Asset'][df_discreteSummaries['Cruise'].str.contains(cruise) & df_discreteSummaries['Cast'].str.contains(cast)].unique()[0]
        if isinstance(targetSites,str):
            targetSites = targetSites.split(',')
        else:
            targetSites = 'undefined'
        targetAssets = []
        for site in targetSites:
            for key in inst_dict:
                if site in key:
                    for inst in inst_dict[key]['instruments'].split(','):
                        targetAssets.append(key + '-' + inst)
                        targetAssetsAll.append(key + '-' + inst)
        ### TODO: find targetAssets based on lat/long location

        for asset in targetAssets:
            sampleTime = dt.datetime.strptime(arrayData[cruise][cast]['sampleTime'], '%Y-%m-%dT%H:%M:%S.%fZ')
            ### find cast window within 14 days of sample time
            castWindows = castWindow(asset,sampleTime,14)
            #print(asset)
            if not castWindows:
                #print('no valid cast for ' + asset)
                next
            else:
                arrayData[cruise][cast].setdefault(asset,{})
                for window in castWindows:
                    arrayData[cruise][cast][asset].setdefault(window[2],{})
                    arrayData[cruise][cast][asset][window[2]]['castWindow'] = [window[0],window[1]]
                

In [ ]:
targetAssetsUnique = np.unique(targetAssetsAll)
len(targetAssetsUnique)

In [ ]:
test = ['CE02SHBP-LJ01D-06-CTDBPN106','CE04OSPS-SF01B-2A-CTDPFA107',
        'CE04OSPS-SF01B-4F-PCO2WA102','CE04OSPS-SF01B-4A-NUTNRA102',
       'CE04OSPS-SF01B-3A-FLORTD104']

#test = ['CE04OSPS-SF01B-4A-NUTNRA102']
#for target in targetAssetsUnique:
for target in test:
    print('target asset: ' + target)
    instClass = target.split('-')[3][0:6]
    if 'PD-DP' in target:
        instClass = instClass + '_DP'
    #print(instClass)
    zarrFile = 'ooi-data/' + target + '-' + stream_dict[instClass]['stream']
    ### LOAD DATA FOR SITE
    fs = s3fs.S3FileSystem(anon=True)
    zarr_store = s3fs.S3Map(zarrFile, s3=fs)
    ds = xr.open_zarr(zarr_store, consolidated=True)
    
    
    ### iterate through arrayData to add sliced data for each cast window
    for key,value in arrayData.items():
        for L1key,L1value in arrayData[key].items():
            for L2key,L2value in arrayData[key][L1key].items():
                if target in L2key:
                    #print('target found')
                    for L3key,L3value in arrayData[key][L1key][L2key].items():
                        if not isinstance(L3key,str):
                            #print('deployment window found')
                            castStart = arrayData[key][L1key][L2key][L3key]['castWindow'][0]
                            castEnd = arrayData[key][L1key][L2key][L3key]['castWindow'][1]
                            params = stream_dict[instClass]['parameters']
                            if 'profiler' in inst_dict[target[0:14]]['platform']:
                                if not any('pressure' in entry for entry in params):
                                    #print('pressure param not found...appending')
                                    params.append('int_ctd_pressure')
                            data = ds[params].sel(time=slice(castStart, castEnd)).load()
                            arrayData[key][L1key][L2key][L3key]['arrayData'] = data
                            if 'fixed' in inst_dict[target[0:14]]['platform']:
                                #print('fixed platform')
                                ### extract sample window 
                                extractData = extractFixed(data,arrayData[key][L1key]['sampleTime'],2)
                            elif 'profiler' in inst_dict[target[0:14]]['platform']:
                                #print('profiling platform')
                                ### extract profiles
                                if 'PCO2' in target:
                                    castDirection = 'DOWN'
                                else:
                                    castDirection = 'UP'
                                if len(data[params[0]]) > 10: 
                                    extractData = extractProfiles(data,arrayData[key][L1key]['sampleTime'],castDirection)
                                else:
                                    extractData = 'NoValidData'
                            arrayData[key][L1key][L2key][L3key]['arrayData'] = extractData
                            #print(key)
                            #print(L1key)
                            #print(L2key)
                            #print(L3key)
    
    ### clear variable to free memory
    del ds
    gc.collect()

In [ ]:
startTime_year = []

for timeEntry in df_discreteSummaries['Start Time [UTC]']:
    startTime = np.datetime64(dt.datetime.strptime(timeEntry,'%Y-%m-%dT%H:%M:%S.%fZ'))
    startTime_year.append(startTime.astype(object).year)

df_discreteSummaries['sampleYear'] = startTime_year

In [ ]:
# Added code to set recursion limit to fix the following error:
# RecursionError: maximum recursion depth exceeded while calling a Python object
#import sys
#sys.setrecursionlimit(150000)
## After running the code once, the kernel crashed.  Ran again, and the below code executed.
## Set recursion limit back to 1500, and code ran fine.  Commented out recursion limit, and code seems
## to be running fine...

import gsw

df_discreteSummaries['Discrete Conductivity [S/m]'] = gsw.C_from_SP(df_discreteSummaries['Discrete Salinity [psu]'],
                                                  df_discreteSummaries['CTD Temperature 1 [deg C]'],
                                                  df_discreteSummaries['CTD Pressure [db]'])/10

df_discreteSummaries['Cond_diff_1'] = df_discreteSummaries['CTD Conductivity 1 [S/m]'] - df_discreteSummaries['Discrete Conductivity [S/m]']
df_discreteSummaries['Cond_diff_2'] = df_discreteSummaries['CTD Conductivity 2 [S/m]'] - df_discreteSummaries['Discrete Conductivity [S/m]']
df_discreteSummaries['Cond_diff_CTD'] = df_discreteSummaries['CTD Conductivity 1 [S/m]'] - df_discreteSummaries['CTD Conductivity 2 [S/m]']


In [ ]:
Axial_df = df_discreteSummaries[df_discreteSummaries['Station'].str.contains('Axial',case=False)]
SlopeBase_df = df_discreteSummaries[df_discreteSummaries['Station'].str.contains('Slope Base',case=False)]
HydrateRidge_df = df_discreteSummaries[df_discreteSummaries['Station'].str.contains('Hydrate Ridge',case=False)]
Offshore_df = df_discreteSummaries[df_discreteSummaries['Station'].str.contains('Offshore',case=False)]
Shelf_df = df_discreteSummaries[df_discreteSummaries['Station'].str.contains('Shelf',case=False)]


In [ ]:
sites = ['Axial','SlopeBase','HydrateRidge','Offshore', 'Shelf']
checkbox_sites = pn.widgets.CheckBoxGroup(
    name='checkbox_sites', value=['Axial'], options=sites,
    inline=False)


In [ ]:
years = ['2015 \u25b2','2016 \u26ab','2017 \u2b25','2018 \u25a0','2019 \u25bc']
#years = ['2015','2016','2017','2018','2019']
checkbox_years = pn.widgets.CheckBoxGroup(
    name='checkbox_years', value=['2015 \u25b2'], options=years,
    inline=False)

In [ ]:
Axial_color = 'dodgerblue'
SlopeBase_color = 'darkred'
HydrateRidge_color = 'darkturquoise'
Offshore_color = 'seagreen'
Shelf_color = 'darkmagenta'

marker_2015 = 'triangle'
marker_2016 = 'circle'
marker_2017 = 'diamond'
marker_2018 = 'square'
marker_2019 = 'inverted_triangle'


In [ ]:
# Tab 1: plot profiles by Year, location

# set a title for your dashboard
title_Tab1 = '## Temperature Discrete Casts'

subtitle_Tab1 = 'This dashboard allows you to select a year to view temperature profiles from discrete casts.'


# tell Panel what your plot "depends" on. 
# This defines what should trigger a change in the chart.
# both values in depends() will be used in our below chart as filters
@pn.depends(checkbox_years, checkbox_sites)
def plot_tab1(checkbox_years,checkbox_sites): # start function
    
    print(checkbox_years)
    print(checkbox_sites)
    
    s1 = figure(width=350, plot_height=350, title = 'CTD Temperature',background_fill_color='whitesmoke')
    s2 = figure(width=350, plot_height=350, title = 'CTD Conductivity',background_fill_color='whitesmoke')
    s3 = figure(width=350, plot_height=350, title = 'CTD Salinity',background_fill_color='whitesmoke')
    s4 = figure(width=350, plot_height=350, title = 'CTD Oxygen',background_fill_color='whitesmoke')
    s5 = figure(width=350, plot_height=350, title = 'CTD Fluorescence',background_fill_color='whitesmoke')
    s6 = figure(width=350, plot_height=350, title = 'CTD Beam Transmission',background_fill_color='whitesmoke')
    s7 = figure(width=350, plot_height=350, title = 'CTD pH',background_fill_color='whitesmoke')
    
                  
    for site in checkbox_sites:
        df = eval(site + '_df')
        for yearEntry in checkbox_years:
            year = (yearEntry[0:4])
            mask = np.ma.masked_equal(df['sampleYear'] == int(year),df)
            plotColor = eval(site + '_color')
            plotMarker = eval('marker_' + year)
            eval('s1.' + plotMarker + "(df['CTD Temperature 1 [deg C]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s2.' + plotMarker + "(df['CTD Conductivity 1 [S/m]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s3.' + plotMarker + "(df['CTD Salinity 1 [psu]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s4.' + plotMarker + "(df['CTD Oxygen [mL/L]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s5.' + plotMarker + "(df['CTD Fluorescence [mg/m^3]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s6.' + plotMarker + "(df['CTD Beam Transmission [%]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s7.' + plotMarker + "(df['CTD pH'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
    
    grid = gridplot([s1, s2, s3, s4, s5, s6, s7], ncols=3)
    
    return grid

# create the Panel object, passing in all smaller objects
tab1_pane = pn.Row(
    pn.Column(title_Tab1, subtitle_Tab1, checkbox_years, checkbox_sites),
    plot_tab1)



In [ ]:
# Tab 2: plot CTD vs Discrete profiles by Year, location

# set a title for your dashboard
title_Tab2 = '## CTD vs Discrete profiles'

subtitle_Tab2 = 'This dashboard allows you to select a year to view CTD vs Discrete profiles.'

@pn.depends(checkbox_years, checkbox_sites)
def plot_tab2(checkbox_years, checkbox_sites): # start function
    
    print(checkbox_years)
    print(checkbox_sites)
    
    s1 = figure(width=350, plot_height=350, title = 'Salinity',background_fill_color='whitesmoke')
    s2 = figure(width=350, plot_height=350, title = 'Oxygen',background_fill_color='whitesmoke')
    s3 = figure(width=350, plot_height=350, title = 'Fluorescence',background_fill_color='whitesmoke')
    s4 = figure(width=350, plot_height=350, title = 'pH',background_fill_color='whitesmoke')
    s5 = figure(width=350, plot_height=350, title = 'CTD Temperature',background_fill_color='whitesmoke')
    s6 = figure(width=350, plot_height=350, title = 'CTD Salinity',background_fill_color='whitesmoke')
    s7 = figure(width=350, plot_height=350, title = 'CTD Conductivity',background_fill_color='whitesmoke')
    
    for site in checkbox_sites:
        df = eval(site + '_df')
        for yearEntry in checkbox_years:
            year = (yearEntry[0:4])
            mask = np.ma.masked_equal(df['sampleYear'] == int(year),df)
            plotColor = eval(site + '_color')
            plotMarker = eval('marker_' + year)
            eval('s1.' + plotMarker + "(df['CTD Salinity 1 [psu]'][mask],df['Discrete Salinity [psu]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s2.' + plotMarker + "(df['CTD Oxygen [mL/L]'][mask],df['Discrete Oxygen [mL/L]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s3.' + plotMarker + "(df['CTD Fluorescence [mg/m^3]'][mask],df['Discrete Chlorophyll [ug/L]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s4.' + plotMarker + "(df['CTD pH'][mask],df['Calculated pH'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s5.' + plotMarker + "(df['CTD Temperature 1 [deg C]'][mask],df['CTD Temperature 2 [deg C]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s6.' + plotMarker + "(df['CTD Salinity 1 [psu]'][mask],df['CTD Salinity 2 [psu]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s7.' + plotMarker + "(df['CTD Conductivity 1 [S/m]'][mask],df['CTD Conductivity 2 [S/m]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
    
    grid = gridplot([s1, s2, s3, s4, s5, s6, s7], ncols=3)
    
    return grid

tab2_pane = pn.Row(
    pn.Column(title_Tab2, subtitle_Tab2, checkbox_years, checkbox_sites),
    plot_tab2 #draw chart function!
)

In [ ]:
# Tab 3: plot discrete samples by Year, location

# set a title for your dashboard
title_Tab3 = '## Discrete Casts'

subtitle_Tab3 = 'This dashboard allows you to select a year to view discrete casts.'


parameters = ['Oxygen','Salinity','Chlorophyll','Nutrients','Carbon']
variable = pn.widgets.RadioButtonGroup(name='variable',value='Oxygen',
                                      options=list(parameters), sizing_mode='scale_width')

showCTDcast = pn.widgets.Checkbox(name='Show CTD cast')

# tell Panel what your plot "depends" on. 
# This defines what should trigger a change in the chart.
# both values in depends() will be used in our below chart as filters
@pn.depends(checkbox_years, checkbox_sites, variable, showCTDcast)
def plot_tab3(checkbox_years,checkbox_sites, variable, showCTDcast): # start function
    
    print(checkbox_years)
    print(checkbox_sites)
    print(variable)
    print(showCTDcast)
    
    if variable == 'Oxygen':
        s1 = figure(width=350, plot_height=350, title = 'Discrete Oxygen',background_fill_color='whitesmoke')
    elif variable == 'Salinity':
        s1 = figure(width=350, plot_height=350, title = 'Discrete Salinity',background_fill_color='whitesmoke')
    elif variable == 'Chlorophyll':
        s1 = figure(width=350, plot_height=350, title = 'Discrete Chlorophyll',background_fill_color='whitesmoke')
        s2 = figure(width=350, plot_height=350, title = 'Discrete Phaeopigment',background_fill_color='whitesmoke')
        s3 = figure(width=350, plot_height=350, title = 'Discrete Fo/Fa Ratio',background_fill_color='whitesmoke')
    elif variable == 'Nutrients':
        s1 = figure(width=350, plot_height=350, title = 'Discrete Phosphate',background_fill_color='whitesmoke')
        s2 = figure(width=350, plot_height=350, title = 'Discrete Silicate',background_fill_color='whitesmoke')
        s3 = figure(width=350, plot_height=350, title = 'Discrete Nitrate',background_fill_color='whitesmoke')
        s4 = figure(width=350, plot_height=350, title = 'Discrete Nitrite',background_fill_color='whitesmoke')
        s5 = figure(width=350, plot_height=350, title = 'Discrete Ammonium',background_fill_color='whitesmoke')
    elif variable == 'Carbon':
        s1 = figure(width=350, plot_height=350, title = 'Discrete DIC',background_fill_color='whitesmoke')
        s2 = figure(width=350, plot_height=350, title = 'Discrete pCO2',background_fill_color='whitesmoke')
        s3 = figure(width=350, plot_height=350, title = 'Calculated Alkalinity',background_fill_color='whitesmoke')
        s4 = figure(width=350, plot_height=350, title = 'Calculated CO2aq ',background_fill_color='whitesmoke')
        s5 = figure(width=350, plot_height=350, title = 'Calculated Bicarb',background_fill_color='whitesmoke')
        s6 = figure(width=350, plot_height=350, title = 'Calculated CO3',background_fill_color='whitesmoke')
        s7 = figure(width=350, plot_height=350, title = 'Calculated Omega-C',background_fill_color='whitesmoke')
        s8 = figure(width=350, plot_height=350, title = 'Calculated Omega-A',background_fill_color='whitesmoke')
        s9 = figure(width=350, plot_height=350, title = 'CTD pH',background_fill_color='whitesmoke')

    for site in checkbox_sites:
        df = eval(site + '_df')
        for yearEntry in checkbox_years:
            year = (yearEntry[0:4])
            mask = np.ma.masked_equal(df['sampleYear'] == int(year),df)
            plotColor = eval(site + '_color')
            plotMarker = eval('marker_' + year)
            if variable == 'Oxygen':
                eval('s1.' + plotMarker + "(df['Discrete Oxygen [mL/L]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                if showCTDcast:
                    eval('s1.' + plotMarker + "(df['CTD Oxygen [mL/L]'][mask],-df['CTD Pressure [db]'][mask],size=5, fill_color=plotColor, line_color='black', line_width=0.25)")
                grid = s1
            elif variable == 'Salinity':
                eval('s1.' + plotMarker + "(df['Discrete Salinity [psu]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                if showCTDcast:
                    eval('s1.' + plotMarker + "(df['CTD Salinity 1 [psu]'][mask],-df['CTD Pressure [db]'][mask],size=5, fill_color=plotColor, line_color='black', line_width=0.25)")
                grid = s1
            elif variable == 'Chlorophyll':
                eval('s1.' + plotMarker + "(df['Discrete Chlorophyll [ug/L]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                if showCTDcast:
                    eval('s1.' + plotMarker + "(df['CTD Fluorescense [mg/m^3]'][mask],-df['CTD Pressure [db]'][mask],size=5, fill_color=plotColor, line_color='black', line_width=0.25)")
                eval('s2.' + plotMarker + "(df['Discrete Phaeopigment [ug/L]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s3.' + plotMarker + "(df['Discrete Fo/Fa Ratio'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                grid = gridplot([s1, s2, s3], ncols=3)
            elif variable == 'Nutrients':
                eval('s1.' + plotMarker + "(df['Discrete Phosphate [uM]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s2.' + plotMarker + "(df['Discrete Silicate [uM]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s3.' + plotMarker + "(df['Discrete Nitrate [uM]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s4.' + plotMarker + "(df['Discrete Nitrite [uM]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s5.' + plotMarker + "(df['Discrete Ammonium [uM]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                grid = gridplot([s1, s2, s3, s4, s5], ncols=3)
            elif variable == 'Carbon':
                eval('s1.' + plotMarker + "(df['Discrete DIC [umol/kg]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s2.' + plotMarker + "(df['Discrete pCO2 [uatm]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")                
                eval('s3.' + plotMarker + "(df['Calculated Alkalinity [umol/kg]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s4.' + plotMarker + "(df['Calculated CO2aq [umol/kg]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s5.' + plotMarker + "(df['Calculated Bicarb [umol/kg]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s6.' + plotMarker + "(df['Calculated CO3 [umol/kg]'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s7.' + plotMarker + "(df['Calculated Omega-C'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s8.' + plotMarker + "(df['Calculated Omega-A'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                eval('s9.' + plotMarker + "(df['Calculated pH'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
                if showCTDcast:
                    eval('s9.' + plotMarker + "(df['CTD pH'][mask],-df['CTD Pressure [db]'][mask],size=5, fill_color=plotColor, line_color='black', line_width=0.25)")
                grid = gridplot([s1, s2, s3, s4, s5, s6, s7, s8, s9], ncols=3)
    return grid

tab3_pane = pn.Row(
    pn.Column(title_Tab3, subtitle_Tab3, checkbox_years, checkbox_sites, variable, showCTDcast),
    plot_tab3 #draw chart function!
)


In [ ]:
# Tab 4: plot profiles by Year, location

# set a title for your dashboard
title_Tab4 = '## Conductivity Corrections'

subtitle_Tab4 = 'This dashboard allows you to select a year to view conductivity.'


# tell Panel what your plot "depends" on. 
# This defines what should trigger a change in the chart.
# both values in depends() will be used in our below chart as filters
@pn.depends(checkbox_years, checkbox_sites)
def plot_tab4(checkbox_years,checkbox_sites): # start function
    
    print(checkbox_years)
    print(checkbox_sites)
    
    s1 = figure(width=350, plot_height=350, title = 'CTD Conductivity diff',background_fill_color='whitesmoke')
    s2 = figure(width=350, plot_height=350, title = 'CTD Conductivity vs Discrete',background_fill_color='whitesmoke')
                     
    for site in checkbox_sites:
        df = eval(site + '_df')
        for yearEntry in checkbox_years:
            year = (yearEntry[0:4])
            mask = np.ma.masked_equal(df['sampleYear'] == int(year),df)
            plotColor = eval(site + '_color')
            plotMarker = eval('marker_' + year)
            eval('s1.' + plotMarker + "(df['Cond_diff_CTD'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s2.' + plotMarker + "(df['Cond_diff_1'][mask],-df['CTD Pressure [db]'][mask],size=5, color=plotColor, fill_alpha=0.25)")
            eval('s2.' + plotMarker + "(df['Cond_diff_2'][mask],-df['CTD Pressure [db]'][mask],size=5, fill_color=plotColor, line_color='black', line_width=0.25)")
           
    grid = gridplot([s1, s2], ncols=2)
    
    return grid

# create the Panel object, passing in all smaller objects
tab4_pane = pn.Row(
    pn.Column(title_Tab4, subtitle_Tab4, checkbox_years, checkbox_sites),
    plot_tab4)


In [ ]:
# Tab 5: plot discrete samples by Year, location

# set a title for your dashboard
title_Tab5 = '## Discrete Casts'

subtitle_Tab5 = 'This dashboard allows you to select a year to view discrete casts.'


parameters = ['Oxygen','Salinity','Chlorophyll','Nitrate','pCO2','pH']
variable = pn.widgets.RadioButtonGroup(name='variable',value='Oxygen',
                                      options=list(parameters), sizing_mode='scale_width')

showCTDcast = pn.widgets.Checkbox(name='Show CTD cast')
showArrayData = pn.widgets.Checkbox(name='Show Array data')

# tell Panel what your plot "depends" on. 
# This defines what should trigger a change in the chart.
# both values in depends() will be used in our below chart as filters
@pn.depends(checkbox_years, checkbox_sites, variable, showCTDcast, showArrayData)
def plot_tab5(checkbox_years,checkbox_sites, variable, showCTDcast, showArrayData): # start function
    
    print(checkbox_years)
    print(checkbox_sites)
    print(variable)
    print(showCTDcast)
    print(showArrayData)
    
    if variable == 'Oxygen':
        s1 = figure(width=750, plot_height=750, title = 'Discrete Oxygen',background_fill_color='whitesmoke')
    elif variable == 'Salinity':
        s1 = figure(width=750, plot_height=750, title = 'Discrete Salinity',background_fill_color='whitesmoke')
    elif variable == 'Chlorophyll':
        s1 = figure(width=750, plot_height=750, title = 'Discrete Chlorophyll',background_fill_color='whitesmoke')
    elif variable == 'Nitrate':
        s1 = figure(width=750, plot_height=750, title = 'Discrete Nitrate',background_fill_color='whitesmoke')
    elif variable == 'pCO2':
        s1 = figure(width=750, plot_height=750, title = 'Discrete pCO2',background_fill_color='whitesmoke')
    elif variable == 'pH':
        s1 = figure(width=750, plot_height=750, title = 'CTD pH',background_fill_color='whitesmoke')

    for site in checkbox_sites:
        for cruise in arrayData:
            for cast in arrayData[cruise]:
                if site in arrayData[cruise][cast]['station']:
                    castTime = np.datetime64(dt.datetime.strptime(arrayData[cruise][cast]['sampleTime'],'%Y-%m-%dT%H:%M:%S.%fZ'))
                    castYear = int(castTime.astype(object).year)
                    for yearEntry in checkbox_years:
                        year = (yearEntry[0:4])
                        if int(castYear) == int(year):
                            if 'dataOnlyCast' not in arrayData[cruise][cast]['bottleData']['castType']:
                                arrayList=list(arrayData[cruise][cast].keys())
                                arrayList = [e for e in arrayList if e not in ('sampleTime', 'station','bottleData')]
                                plotColor = eval(site + '_color')
                                plotMarker = eval('marker_' + year)
                                CTDpress_ = arrayData[cruise][cast]['bottleData']['CTD Pressure [db]']
                                CTDpress = [ -x for x in CTDpress_]
                                
                                if variable == 'Oxygen':
                                    eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['Discrete Oxygen [mL/L]'],CTDpress,size=5, color=plotColor, fill_alpha=0.25)")
                                    if showCTDcast:
                                        eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['CTD Oxygen [mL/L]'],CTDpress,size=3, fill_color=plotColor, line_color='black', line_width=0.25)")
                                        s1.line(arrayData[cruise][cast]['bottleData']['CTD Oxygen [mL/L]'],CTDpress, line_width=1, color=plotColor)
                                    if showArrayData:
                                        subList = [n for n in arrayList if 'CTD' in n]
                                        for refDes in subList:
                                            for deploy in arrayData[cruise][cast][refDes]:
                                                if 'arrayData' in arrayData[cruise][cast][refDes][deploy]:
                                                    if 'NoValidData' not in arrayData[cruise][cast][refDes][deploy]['arrayData']:
                                                        arrayTime = max(arrayData[cruise][cast][refDes][deploy]['arrayData']['time'].values)
                                                        variableData = arrayData[cruise][cast][refDes][deploy]['arrayData']['corrected_dissolved_oxygen'].values * .022391
                                                        pressData_ = arrayData[cruise][cast][refDes][deploy]['arrayData']['seawater_pressure'].values
                                                        pressData = [ -x for x in pressData_]
                                                        if arrayTime > castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dotted')
                                                        elif arrayTime < castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dashed')                        
                        
                                elif variable == 'Salinity':
                                    eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['Discrete Salinity [psu]'],CTDpress,size=5, color=plotColor, fill_alpha=0.25)")
                                    if showCTDcast:
                                        eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['CTD Salinity 1 [psu]'],CTDpress,size=3, fill_color=plotColor, line_color='black', line_width=0.25)")
                                        eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['CTD Salinity 2 [psu]'],CTDpress,size=3, fill_color=plotColor, line_color='black', line_width=0.25)")
                                        s1.line(arrayData[cruise][cast]['bottleData']['CTD Salinity 1 [psu]'],CTDpress, line_width=1, color=plotColor)
                                        s1.line(arrayData[cruise][cast]['bottleData']['CTD Salinity 2 [psu]'],CTDpress, line_width=1, color=plotColor)
                                    if showArrayData:
                                        subList = [n for n in arrayList if 'CTD' in n]
                                        for refDes in subList:
                                            for deploy in arrayData[cruise][cast][refDes]:
                                                if 'arrayData' in arrayData[cruise][cast][refDes][deploy]:
                                                    if 'NoValidData' not in arrayData[cruise][cast][refDes][deploy]['arrayData']:
                                                        arrayTime = max(arrayData[cruise][cast][refDes][deploy]['arrayData']['time'].values)
                                                        variableData = arrayData[cruise][cast][refDes][deploy]['arrayData']['practical_salinity'].values
                                                        pressData_ = arrayData[cruise][cast][refDes][deploy]['arrayData']['seawater_pressure'].values
                                                        pressData = [ -x for x in pressData_]
                                                        if arrayTime > castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dotted')
                                                        elif arrayTime < castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dashed')
                                
                                elif variable == 'Chlorophyll':
                                    eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['Discrete Chlorophyll [ug/L]'],CTDpress,size=5, color=plotColor, fill_alpha=0.25)")
                                    if showCTDcast:
                                        eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['CTD Fluorescence [mg/m^3]'],CTDpress,size=3, fill_color=plotColor, line_color='black', line_width=0.25)")
                                        s1.line(arrayData[cruise][cast]['bottleData']['CTD Fluorescence [mg/m^3]'],CTDpress, line_width=1, color=plotColor)
                                    if showArrayData:
                                        subList = [n for n in arrayList if 'FL' in n]
                                        for refDes in subList:
                                            for deploy in arrayData[cruise][cast][refDes]:
                                                if 'arrayData' in arrayData[cruise][cast][refDes][deploy]:
                                                    if 'NoValidData' not in arrayData[cruise][cast][refDes][deploy]['arrayData']:
                                                        arrayTime = max(arrayData[cruise][cast][refDes][deploy]['arrayData']['time'].values)
                                                        variableData = arrayData[cruise][cast][refDes][deploy]['arrayData']['fluorometric_chlorophyll_a'].values
                                                        pressData_ = arrayData[cruise][cast][refDes][deploy]['arrayData']['int_ctd_pressure'].values
                                                        pressData = [ -x for x in pressData_]
                                                        if arrayTime > castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dotted')
                                                        elif arrayTime < castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dashed')

                                elif variable == 'Nitrate':
                                    eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['Discrete Nitrate [uM]'],CTDpress,size=5, color=plotColor, fill_alpha=0.25)")
                                    if showArrayData:
                                        subList = [n for n in arrayList if 'NUT' in n]
                                        for refDes in subList:
                                            for deploy in arrayData[cruise][cast][refDes]:
                                                if 'arrayData' in arrayData[cruise][cast][refDes][deploy]:
                                                    if 'NoValidData' not in arrayData[cruise][cast][refDes][deploy]['arrayData']:
                                                        arrayTime = max(arrayData[cruise][cast][refDes][deploy]['arrayData']['time'].values)
                                                        variableData = arrayData[cruise][cast][refDes][deploy]['arrayData']['salinity_corrected_nitrate'].values
                                                        pressData_ = arrayData[cruise][cast][refDes][deploy]['arrayData']['int_ctd_pressure'].values
                                                        pressData = [ -x for x in pressData_]
                                                        if arrayTime > castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dotted')
                                                        elif arrayTime < castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dashed')

                                elif variable == 'pCO2':
                                    eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['Discrete pCO2 [uatm]'],CTDpress,size=5, color=plotColor, fill_alpha=0.25)")
                                    if showArrayData:
                                        subList = [n for n in arrayList if 'PCO2' in n]
                                        for refDes in subList:
                                            for deploy in arrayData[cruise][cast][refDes]:
                                                if 'arrayData' in arrayData[cruise][cast][refDes][deploy]:
                                                    if 'NoValidData' not in arrayData[cruise][cast][refDes][deploy]['arrayData']:
                                                        arrayTime = max(arrayData[cruise][cast][refDes][deploy]['arrayData']['time'].values)
                                                        variableData = arrayData[cruise][cast][refDes][deploy]['arrayData']['pco2_seawater'].values
                                                        pressData_ = arrayData[cruise][cast][refDes][deploy]['arrayData']['int_ctd_pressure'].values
                                                        pressData = [ -x for x in pressData_]
                                                        if arrayTime > castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dotted')
                                                        elif arrayTime < castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dashed')

                                elif variable == 'pH':
                                    eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['Calculated pH'],CTDpress,size=5, color=plotColor, fill_alpha=0.25)")
                                    if showCTDcast:
                                        eval('s1.' + plotMarker + "(arrayData[cruise][cast]['bottleData']['CTD pH'],CTDpress,size=3, fill_color=plotColor, line_color='black', line_width=0.25)")
                                        s1.line(arrayData[cruise][cast]['bottleData']['CTD pH'],CTDpress, line_width=1, color=plotColor)
                                    if showArrayData:
                                        subList = [n for n in arrayList if 'PHSEN' in n]
                                        for refDes in subList:
                                            for deploy in arrayData[cruise][cast][refDes]:
                                                if 'arrayData' in arrayData[cruise][cast][refDes][deploy]:
                                                    if 'NoValidData' not in arrayData[cruise][cast][refDes][deploy]['arrayData']:
                                                        arrayTime = max(arrayData[cruise][cast][refDes][deploy]['arrayData']['time'].values)
                                                        variableData = arrayData[cruise][cast][refDes][deploy]['arrayData']['ph_seawater'].values
                                                        pressData_ = arrayData[cruise][cast][refDes][deploy]['arrayData']['int_ctd_pressure'].values
                                                        pressData = [ -x for x in pressData_]
                                                        if arrayTime > castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dotted')
                                                        elif arrayTime < castTime:
                                                            s1.line(variableData,pressData,line_width=1, color=plotColor, line_dash = 'dashed')

    grid = s1   
    return grid

tab5_pane = pn.Row(
    pn.Column(title_Tab5, subtitle_Tab5, checkbox_years, checkbox_sites, variable, showCTDcast, showArrayData),
    plot_tab5 #draw chart function!
)




#'CE04OSPD-DP01B-06-DOSTAD105', 



In [ ]:
pn.Tabs(
    ('CTD Profiles', tab1_pane),
    ('CTD vs', tab2_pane),
    ('Discrete', tab3_pane),
    ('Conductivity', tab4_pane),
    ('Array', tab5_pane),
    dynamic=True
).servable()

In [ ]:
# in terminal enter:
#python3 -m panel serve --show discreteDashboard-CurrentDraft_AWS.ipynb